# Automated CapsnetsS2I train + test

First, import stuff

In [8]:
import model_s2i
import data_loader
import tensorflow as tf

In [9]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Load word vectors and data

In [10]:
WORD_VEC_SCENARIO = "FastText-300-verbsDatasets-synAntVSP-dia"
SCENARIO_NAME = 'no-attention'

word2vec_path = '../../romanian_word_vecs/cleaned-vectors.vec'

# No diacritics data
training_data_paths = [
#         '../data-capsnets/scenario0/train.txt',
#         '../data-capsnets/scenario1/train.txt',
#         '../data-capsnets/scenario2/train.txt',
#         '../data-capsnets/scenario3.1/train.txt',
#         '../data-capsnets/scenario3.2/train.txt',
#         '../data-capsnets/scenario3.3/train.txt',
        '../data-capsnets/diacritics/scenario0/train.txt',
        '../data-capsnets/diacritics/scenario1/train.txt',
        '../data-capsnets/diacritics/scenario2/train.txt',
        '../data-capsnets/diacritics/scenario31/train.txt',
        '../data-capsnets/diacritics/scenario32/train.txt',
        '../data-capsnets/diacritics/scenario33/train.txt',
        
    ]
test_data_paths = [
#     '../data-capsnets/scenario0/test.txt',
#     '../data-capsnets/scenario1/test.txt',
#     '../data-capsnets/scenario2/test.txt',
#     '../data-capsnets/scenario3.1/test.txt',
#     '../data-capsnets/scenario3.2/test.txt',
#     '../data-capsnets/scenario3.3/test.txt',
    '../data-capsnets/diacritics/scenario0/test.txt',
    '../data-capsnets/diacritics/scenario1/test.txt',
    '../data-capsnets/diacritics/scenario2/test.txt',
    '../data-capsnets/diacritics/scenario31/test.txt',
    '../data-capsnets/diacritics/scenario32/test.txt',
    '../data-capsnets/diacritics/scenario33/test.txt',
]

scenario_nums = [
#     '0_' + WORD_VEC_SCENARIO,
#     '1_' + WORD_VEC_SCENARIO,
#     '2_' + WORD_VEC_SCENARIO,
#     '31_' + WORD_VEC_SCENARIO,
#     '32_' + WORD_VEC_SCENARIO,
#     '33_' + WORD_VEC_SCENARIO,
#     '0_dia_' + WORD_VEC_SCENARIO,
#     '1_dia_' + WORD_VEC_SCENARIO,
#     '2_dia_' + WORD_VEC_SCENARIO,
#     '31_dia_' + WORD_VEC_SCENARIO,
#     '32_dia_' + WORD_VEC_SCENARIO,
#     '33_dia_' + WORD_VEC_SCENARIO,
    '0-' + SCENARIO_NAME,
    '1-' + SCENARIO_NAME,
    '2-' + SCENARIO_NAME,
    '31-' + SCENARIO_NAME,
    '32-' + SCENARIO_NAME,
    '33-' + SCENARIO_NAME,
]

In [11]:
print('------------------load word2vec begin-------------------')
w2v = data_loader.load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
loading time took 000.06
------------------load word2vec end---------------------


# Flag setting functions + utils

In [12]:
import csv

def set_flags(data, scenario):
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
    tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
    tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
    tf.app.flags.DEFINE_string('scenario_num', scenario, 'Scenario number')
    tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
    tf.app.flags.DEFINE_string('results_dir', './results/', 'Results dir')
    tf.app.flags.DEFINE_string('hyperparams_dir', './hyperparams/', 'Hyperparams dir')
    
    vocab_size, word_emb_size = data['embedding'].shape
    _, max_sentence_length = data['x_tr'].shape
    intents_number = len(data['intents_dict'])
    slots_number = len(data['slots_dict'])
    hidden_size = 64

    tf.app.flags.DEFINE_float('keep_prob', 1, 'embedding dropout keep rate for training')
    tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
    tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
    tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
    tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
    tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
    tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
    tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
    tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')
    tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
    tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
    tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
    tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
    tf.app.flags.DEFINE_integer('intent_output_dim', 16, 'intent output dimension')
    tf.app.flags.DEFINE_integer('slot_output_dim', 32, 'slot output dimension')
    tf.app.flags.DEFINE_integer('d_a', 20, 'self attention weight hidden units number')
    tf.app.flags.DEFINE_integer('r', 5, 'number of self attention heads')
    tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
    tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
    tf.app.flags.DEFINE_float('rerouting_coef', 0.5, 'coefficient for rerouting')
    tf.app.flags.DEFINE_boolean('use_rerouting', True, 'whether to use rerouting or not')
    tf.app.flags.DEFINE_boolean('use_attention', False,
                                'whether to use attention or not. If attention is used,'
                                'slot_output_dim will be overridden to hidden_size * 2')
    
    tf.app.flags.DEFINE_string('f', '', 'kernel') # Without this there's an error: unknown command line flag 'f'
    
    return FLAGS

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
def write_csv(run_results):
    with open('results/results_{}.csv'.format(SCENARIO_NAME), 'w') as f:
        header_line = ['Scenario', 'Intent F1', 'Slot F1']
        writer = csv.writer(f)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            intent_score = [v['intent_f1']]
            slot_score = [v['slot_f1']]
            l = scenario_num + intent_score + slot_score
            writer.writerow(l)

In [6]:
# In case it's needed
del_all_flags(FLAGS)

NameError: name 'FLAGS' is not defined

In [13]:
import train
import test

results = dict()
for i in range(len(training_data_paths)):
    
    # Load data
    data = data_loader.read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True)
    
    FLAGS = set_flags(data, scenario=scenario_nums[i])
    
    # Train model
    print('TRAINING ' + scenario_nums[i])
    train.train(model_s2i.CapsNetS2I, data, FLAGS, batches_rand=False)
    
    # Test model
    print('TESTING ' + scenario_nums[i])
    intent_f, slot_f = test.test(model_s2i.CapsNetS2I, data, FLAGS)
    results[scenario_nums[i]] = dict()
    results[scenario_nums[i]]['intent_f1'] = intent_f
    results[scenario_nums[i]]['slot_f1'] = slot_f
        
    # Reset flags
    del_all_flags(FLAGS)

write_csv(results)

------------------read datasets begin-------------------
max length is 15
max length is 15
------------------read datasets end---------------------
TRAINING 0-no-attention
------------------start cross-validation-------------------
FOLD 1
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.071429
F score 0.071429
Slot filling
F1 score: 0.022995
Accuracy: 0.018889
------------------epoch :  0  Loss:  30.56313 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.188095
F score 0.188095
Slot filling
F1 score: 0.196237
Accuracy: 0.896508
Current F score mean 0.19216589861751152
Best F score mean 0.19216589861751152
------------------epoch :  1  Loss:  14.240431 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.423810
F score 0.423810
Slot filling
F1 s

------------------epoch :  5  Loss:  0.60799205 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.900000
F score 0.900000
Slot filling
F1 score: 0.982776
Accuracy: 0.998413
Current F score mean 0.941388044579534
Best F score mean 0.9758399065147532
------------------epoch :  6  Loss:  0.07119053 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.928571
F score 0.928571
Slot filling
F1 score: 0.966700
Accuracy: 0.997778
Current F score mean 0.9476358656479746
Best F score mean 0.9758399065147532
------------------epoch :  7  Loss:  0.03191539 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.952381
F score 0.952381
Slot filling
F1 score: 0.984833
Accuracy: 0.998571
Current F score mean 0.9686070585969473
Best F score mean 0.9758399065147532
------------------epoch :  8  Loss:  0.013921296 ----------------------
           VALIDA

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.984864
Accuracy: 0.999048
Current F score mean 0.9864795060304647
Best F score mean 0.9904943195552333
------------------epoch :  12  Loss:  0.0050719855 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.984864
Accuracy: 0.999048
Current F score mean 0.9864795060304647
Best F score mean 0.9904943195552333
------------------epoch :  13  Loss:  0.004172496 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.988095
F score 0.988095
Slot filling
F1 score: 0.984864
Accuracy: 0.999048
Current F score mean 0.9864795060304647
Best F score mean 0.9904943195552333
------------------epoch :  14  Loss:  0.0035247565 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

...

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.944444
F score 0.944444
Slot filling
F1 score: 0.968230
Accuracy: 0.994949
Current F score mean 0.9563371995293326
Best F score mean 0.9690760059612519
------------------epoch :  12  Loss:  0.43981194 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.964646
F score 0.964646
Slot filling
F1 score: 0.971256
Accuracy: 0.996032
Current F score mean 0.9679510689344275
Best F score mean 0.9690760059612519
------------------epoch :  13  Loss:  0.32788303 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.974747
F score 0.974747
Slot filling
F1 score: 0.974281
Accuracy: 0.996032
Current F score mean 0.9745144332890172
Best F score mean 0.9745144332890172
------------------epoch :  14  Loss:  0.26219502 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.9595

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

Intent F1: 0.271914
Slot F1: 0.575707
